Start modal p3te

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import numpy as np
import os
import cv2
from keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.callbacks import TensorBoard
import tensorflow as tf


connect to google drive

In [ ]:
from google.colab import drive
root = '/content/gdrive/'
drive.mount( root )

Mounted at /content/gdrive/


load train data

In [ ]:
path1='/content/gdrive/MyDrive/NewP3te/train'
all_images  = []
y_train     = []
subjects    = os.listdir(path1)
num_classes = len(subjects)     
print('Number of Classes: ', num_classes)
for number1 in range(0, num_classes):
    print(number1)
    path2 = (path1 + '/' + subjects[number1])
    files = os.listdir(path2)
    numberOfsamples = len(files)
    for number2 in range(0, numberOfsamples):
        path3 = path2 + '/' + files[number2]
        img = cv2.imread(path3)
        img = img.reshape(50, 50, 3)
        all_images.append(img)
        y_train.append(int(number1))

X_train = np.array(all_images)
y_train = np.array(y_train, dtype=np.uint16) 
print(X_train.shape)

Number of Classes:  2
0
1
(5007, 50, 50, 3)


load test data

In [ ]:
path11='/content/gdrive/MyDrive/NewP3te/test'
all_images = []
y_test     = []
subjects   = os.listdir(path11)
for number1 in range(0, num_classes):
    print(number1)
    path2 = (path11 + '/' + subjects[number1])
    files = os.listdir(path2);
    numberOfsamples = len(files)
    for number2 in range(0, numberOfsamples):
        path3 = path2 + '/' + files[number2]
        img = cv2.imread(path3)
        img = img.reshape(50, 50, 3)
        all_images.append(img)
        y_test.append(int(number1))

X_test = np.array(all_images)
y_test = np.array(y_test, dtype=np.uint16) 
print(X_test.shape)

0
1
(634, 50, 50, 3)


process data and normalization

run model

In [ ]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# one hot 
Y_train = tf.keras.utils.to_categorical(y_train , num_classes)
Y_test = tf.keras.utils.to_categorical(y_test , num_classes)
print(X_train.shape, Y_train.shape)

(5007, 50, 50, 3) (5007, 2)


Create model Resnet or ...


In [ ]:
input = tf.keras.Input(shape=(50,50,3))
# efnet = tf.keras.applications.ResNet50(weights='imagenet',
                                             # include_top = False, 
                                             # input_tensor = input)
efnet = tf.keras.applications.efficientnet.EfficientNetB0(weights='imagenet',include_top = False,input_tensor=input)
# Now that we apply global max pooling.
gap = tf.keras.layers.GlobalMaxPooling2D()(efnet.output)

# Finally, we add a classification layer.
output = tf.keras.layers.Dense(2, activation='softmax', use_bias=True)(gap)

# bind all
func_model = tf.keras.Model(efnet.input, output)
func_model.summary()

16719872/16705208 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 50, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 50, 50, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 50, 50, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 51, 51, 3)    0           normalization[0][0]              
_____________________________

In [ ]:
lr_reducer=ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001,patience=10)
save_checkpoint = ModelCheckpoint('/content/gdrive/MyDrive/NewP3te/opmodels-{epoch:02d}.h5', monitor='val_acc',save_best_only=False)
func_model.compile(
          loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])
# fit 
func_model.fit(X_train, Y_train, 
          validation_data=(X_test, Y_test),
          epochs=50, batch_size=64,
          verbose=1,callbacks=[lr_reducer,early_stopper,save_checkpoint])

Epoch 1/50
79/79 [==============================] - 55s 153ms/step - loss: 0.2588 - accuracy: 0.9107 - val_loss: 0.9969 - val_accuracy: 0.6514


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
79/79 [==============================] - 9s 116ms/step - loss: 0.0867 - accuracy: 0.9698 - val_loss: 0.6498 - val_accuracy: 0.6514
Epoch 3/50
79/79 [==============================] - 9s 118ms/step - loss: 0.0501 - accuracy: 0.9824 - val_loss: 2.0292 - val_accuracy: 0.3486
Epoch 4/50
79/79 [==============================] - 9s 118ms/step - loss: 0.0261 - accuracy: 0.9908 - val_loss: 0.7519 - val_accuracy: 0.3391
Epoch 5/50
79/79 [==============================] - 9s 117ms/step - loss: 0.0229 - accuracy: 0.9920 - val_loss: 1.2524 - val_accuracy: 0.3454
Epoch 6/50
79/79 [==============================] - 9s 117ms/step - loss: 0.0215 - accuracy: 0.9914 - val_loss: 3.5477 - val_accuracy: 0.3486
Epoch 7/50
79/79 [==============================] - 9s 116ms/step - loss: 0.0226 - accuracy: 0.9932 - val_loss: 1.0416 - val_accuracy: 0.3596
Epoch 8/50
79/79 [==============================] - 9s 117ms/step - loss: 0.0095 - accuracy: 0.9970 - val_loss: 0.7318 - val_accuracy: 0.4953
Epoch 